In [87]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io

In [67]:
songs = pd.read_csv('data/drake-songs.csv')

In [110]:
song = songs.iloc[0]['lyrics']

In [122]:
summ_words = 0
count_words = 0
summ_chars = 0
count_chars = 0


for line in song.split('|-|'):
    summ_words = summ_words + len(line.split())
    count_words = count_words + 1
    
    summ_chars = summ_chars + len(line)
    count_chars = count_chars + 1

In [123]:
summ_words/count_words

9.07865168539326

In [124]:
summ_chars/count_chars

42.92134831460674

In [68]:
text = ''

for index, row in songs['lyrics'].iteritems():
    text = text + str(row).lower().replace('|-|', '\n')
    
len(text)

374611

In [81]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 54


In [82]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 124857


In [83]:
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [93]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

Build model...


In [94]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               93696     
_________________________________________________________________
dense_3 (Dense)              (None, 54)                6966      
_________________________________________________________________
activation_3 (Activation)    (None, 54)                0         
Total params: 100,662
Trainable params: 100,662
Non-trainable params: 0
_________________________________________________________________


In [95]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [96]:
def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [97]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(
    x, 
    y,
    batch_size=128,
    epochs=30,
    callbacks=[print_callback]
)

Epoch 1/30
124857/124857 [==============================] - 124s 994us/step - loss: 2.0391
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y, nah
never let these niggas ride your "
y, nah
never let these niggas ride your shit it the cand on the say
i gotta shit in my mind on the cand that i got the candes
i gotta spit i me fuck and the cande
i got and the cands it
i got the fucking that i gotta spit it that i got the cand that i gotta scarted that i gotta that i gotta shit it it shit i got the cand it betting for the cand but it that i got the cands cand that i say it that you can't go to the cande that i got and 
----- diversity: 0.5
----- Generating with seed: "y, nah
never let these niggas ride your "
y, nah
never let these niggas ride your back fuck all me cante i should to to you got of the that i have now that i spould the cand
i all me cans
but i wan' put thing girl cands it and go and you say, you can't surting to that but i saying cand go

and i know the money be my ecee friends all back and niggas on my place
damn, you still for the hain for me
i said i'm a mighand and still fuckin' love on my mild
i go niggas party neve
Epoch 8/30
124857/124857 [==============================] - 126s 1ms/step - loss: 1.3567
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "jay and chubbo, shout to gibo
we got san"
jay and chubbo, shout to gibo
we got sant the same and we gone
and i'm a real you get the carse and the six
i need someone that i got a stang
we gon' tire the street things
i don't have to be the start the bitches
i got a lotta like the start to my posile
i got the care that shit the same and the same and the way the carse and the same that i don't want you wanna wanna go
the start is like the best that shit and the same and the start t
----- diversity: 0.5
----- Generating with seed: "jay and chubbo, shout to gibo
we got san"
jay and chubbo, shout to gibo
we got sant go wanna don't really

i'm like my legge start to man
you don't wanna be at a city
you know it better deal every right now
i'm on, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, it's okay, 
Epoch 15/30
124857/124857 [==============================] - 106s 850us/step - loss: 1.2759
----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "overed bria
landmarks of the muses that "
overed bria
landmarks of the muses that i got a good to the close
they gon' take it to the cloties
i got the city to the same all the things
i got the city the shit things it all my lace
and i got the city to the man
you don't take my life
i say you got my love to me
i got the city to the same it and i can't even never still like the way
but i got the shit be the mind the things
i got the city to the same all the

i don't change that 
----- diversity: 0.5
----- Generating with seed: "te, i dedicate, oh yeah
i dedicate, i de"
te, i dedicate, oh yeah
i dedicate, i dedicate, i'll run it
i said i can't even my in the rans story and don't hope the come that hand
you can't even pobod when you know
they all in my in a bitch with
better than we in my problem from the straight
i'm talk and i'm a lot
passing and i'm and the persaling
pasil i seen the way that was been they know
you're too much it with the son somethin'
i got the people that i say
a word man feel a cl
Epoch 22/30
124857/124857 [==============================] - 107s 859us/step - loss: 1.2316
----- Generating text after Epoch: 21
----- diversity: 0.2
----- Generating with seed: " for that
this been where you find me at"
 for that
this been where you find me at mines
i don't even even take the same all the streets i don't even even trust
i be the plug, all i know it all i swear than i don't even was a real
i tell you will we got the same and

/home/nikolaevra/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


a really if i really bitch
i'm on in my city on the city
i got to the creal you wastfucking on the love to long it i say
i got a decise, two languach a preaching how i'm real
and i been through of the street ass in the citys over my own
it's okay, i'm on, i'm on, i'm on, i'm on, i'm on, i'm on, i'm on, lin
i'm on a little me o
Epoch 24/30
124857/124857 [==============================] - 105s 840us/step - loss: 1.2301
----- Generating text after Epoch: 23
----- diversity: 0.2
----- Generating with seed: "n the beach
then its adiios
(killa)
just"
n the beach
then its adiios
(killa)
just the same and i need some really just the way that she wanna don't got me on the sout
you don't wanna done it too much
better find my love
you say it that i got a start to me and i had to
i got the sout that shit to the houding some that i got my love
you know it i wanna be atjuirten the sout it i don't wanna be the way don't take the sould
i got the bent of my niggas is home
i got a desix out my 
----- di

s like you went on vacation with no plan be as a cood
and i need to to some see
they got a reason i wring to be friends.
y'll need to take the club city the things
i know you know it for the beat, the stay the man
i want the six now you could know what i'm a stand to the way too much it
work to the summerstions
i don't wanna be the things in the things that you could lose to grow
show you wanna bleaked at niggas been the whole thing
the


In [99]:
def generate_output():
    generated = ''
    #sentence = text[start_index: start_index + Tx]
    #sentence = '0'*Tx
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 0.2)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

In [100]:
Tx = 40
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: yo, i'm from the six


Here is your poem: 

yo, i'm from the six
tring of the summers the things that i can't even got to the same
you could run and one to me
i got the way the things that i'm a really give and give all my niggas
i got the summersce, man, i got the straight of the stay
you can take my love
and i need to get to the same and i can tell the sicted
i know the streety, i got the way that i got the summersce
i got the summerscy, now the hoes i see i

In [102]:
import utils

In [105]:
utils.save_model_weights(model)

saved model
saved weights
